In [ ]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_timestamp

# Create a Spark session
spark = SparkSession.builder.appName("CustomerTable").getOrCreate()

# Retrieve MySQL credentials from environment variables
mysql_user = os.environ.get("MYSQL_USER")
mysql_password = os.environ.get("MYSQL_PASSWORD")

# Create a DataFrame to represent the customers table with 'created_at' and 'updated_at' columns
customers_data = [
    ("1", "Scott", "Haines", "scott@coffeeco.com"),
    ("2", "John", "Hamm", "john.hamm@acme.com"),
    ("3", "Milo", "Haines", "mhaines@coffeeco.com")
]

columns = ["id", "first_name", "last_name", "email"]
customers_df = spark.createDataFrame(customers_data, columns)

# Add 'created_at' and 'updated_at' columns with default values
customers_df = customers_df.withColumn("created_at", current_timestamp())
customers_df = customers_df.withColumn("updated_at", current_timestamp())

# Configure the MySQL JDBC URL
mysql_url = "jdbc:mysql://localhost:3306/default"

# Set the MySQL credentials using the environment variables
mysql_properties = {
    "user": mysql_user,
    "password": mysql_password,
    "driver": "com.mysql.jdbc.Driver"
}

# Write the DataFrame to the MySQL database
customers_df.write.jdbc(url=mysql_url, table="customers", mode="overwrite", properties=mysql_properties)